In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import fitbit
import gather_keys_oauth2 as Oauth2
import pandas as pd 
import datetime


CLIENT_ID='23QQNX'
CLIENT_SECRET='2cf8156ff7b81eadd26d80a30082067c'

In [2]:
server=Oauth2.OAuth2Server(CLIENT_ID, CLIENT_SECRET)
server.browser_authorize()
ACCESS_TOKEN=str(server.fitbit.client.session.token['access_token'])
REFRESH_TOKEN=str(server.fitbit.client.session.token['refresh_token'])
auth2_client=fitbit.Fitbit(CLIENT_ID,CLIENT_SECRET,oauth2=True,access_token=ACCESS_TOKEN,refresh_token=REFRESH_TOKEN)

[12/Apr/2023:10:40:43] ENGINE Listening for SIGTERM.
[12/Apr/2023:10:40:43] ENGINE Bus STARTING
[12/Apr/2023:10:40:43] ENGINE Set handler for console events.
CherryPy Checker:
The Application mounted at '' has an empty config.

[12/Apr/2023:10:40:43] ENGINE Started monitor thread 'Autoreloader'.
[12/Apr/2023:10:40:44] ENGINE Serving on http://127.0.0.1:8080
[12/Apr/2023:10:40:44] ENGINE Bus STARTED


127.0.0.1 - - [12/Apr/2023:10:40:46] "GET /?code=8f57e5ab8ebc56185b7bab2bcd2fa7115c09a992&state=DOUVPuScHWvfZl5HLq2mucDL8iyTfE HTTP/1.1" 200 122 "" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36"


[12/Apr/2023:10:40:47] ENGINE Bus STOPPING
[12/Apr/2023:10:40:47] ENGINE HTTP Server cherrypy._cpwsgi_server.CPWSGIServer(('127.0.0.1', 8080)) shut down
[12/Apr/2023:10:40:47] ENGINE Removed handler for console events.
[12/Apr/2023:10:40:47] ENGINE Stopped thread 'Autoreloader'.
[12/Apr/2023:10:40:47] ENGINE Bus STOPPED
[12/Apr/2023:10:40:47] ENGINE Bus EXITING
[12/Apr/2023:10:40:47] ENGINE Bus EXITED
[12/Apr/2023:10:40:47] ENGINE Waiting for child threads to terminate...


In [107]:
import requests
import json
import pandas as pd
from time import sleep
from datetime import datetime


def df_fitbit_1(activity, base_date, end_date, token):
    url = 'https://api.fitbit.com/1/user/-/' + activity + '/date/' + base_date + '/' + end_date + '.json'
    response = requests.get(url=url, headers={'Authorization':'Bearer ' + token}).json()

    return response

def df_fitbit_2(token):
    url = 'https://api.fitbit.com/1/user/-/activities/goals/weekly.json'    
    response = requests.get(url=url, headers={'Authorization':'Bearer ' + token}).json()
    return response

def df_fitbit_3(base_date, token):
    url = 'https://api.fitbit.com/1/user/-/activities/date/' + base_date + '.json'
    response = requests.get(url=url, headers={'Authorization':'Bearer ' + token}).json()

    return response


In [113]:
token = ACCESS_TOKEN
import datetime
base_date = '2023-03-29'
end_date = (datetime.datetime.today() - datetime.timedelta(days=1)).strftime('%Y-%m-%d')

activity = ['activities/log/steps', 'activities/log/distance', 'activities/log/calories', 'activities/log/floors', 
            'activities/log/elevation', 'activities/minutesSedentary', 'activities/log/minutesLightlyActive', 
            'activities/log/minutesFairlyActive', 'activities/log/minutesVeryActive', 'activities/log/activityCalories', 
            'sleep/startTime', 'sleep/timeInBed', 'sleep/minutesAsleep', 'sleep/awakeningsCount', 'sleep/minutesAwake', 
            'sleep/minutesToFallAsleep', 'sleep/minutesAfterWakeup', 'sleep/efficiency']

In [77]:
df_fitbit_2(token)

{'goals': {'activeZoneMinutes': 150,
  'caloriesOut': 16541,
  'distance': 56.33,
  'floors': 70,
  'steps': 70000}}

In [110]:
result = df_fitbit_3(base_date, token)
result['goals']

{'activeMinutes': 30,
 'caloriesOut': 2363,
 'distance': 8.05,
 'floors': 10,
 'steps': 10000}

In [111]:
allDates = pd.date_range(start=base_date, end = end_date)

for oneDate in allDates:
    oneDate = oneDate.date().strftime("%Y-%m-%d")
    oneDay = df_fitbit_3(oneDate, token)
    oneDayGoals = oneDay['goals']
    print(oneDate)
    print(oneDayGoals)

2023-03-29
{'activeMinutes': 30, 'caloriesOut': 2363, 'distance': 8.05, 'floors': 10, 'steps': 10000}
2023-03-30
{'activeMinutes': 30, 'caloriesOut': 2363, 'distance': 8.05, 'floors': 10, 'steps': 10000}
2023-03-31
{'activeMinutes': 30, 'caloriesOut': 2363, 'distance': 8.05, 'floors': 10, 'steps': 10000}
2023-04-01
{'activeMinutes': 30, 'caloriesOut': 2363, 'distance': 8.05, 'floors': 10, 'steps': 10000}
2023-04-02
{'activeMinutes': 30, 'caloriesOut': 2363, 'distance': 8.05, 'floors': 10, 'steps': 10000}
2023-04-03
{'activeMinutes': 30, 'caloriesOut': 2363, 'distance': 8.05, 'floors': 10, 'steps': 10000}
2023-04-04
{'activeMinutes': 30, 'caloriesOut': 2363, 'distance': 8.05, 'floors': 10, 'steps': 10000}
2023-04-05
{'activeMinutes': 30, 'caloriesOut': 2363, 'distance': 8.05, 'floors': 10, 'steps': 10000}
2023-04-06
{'activeMinutes': 30, 'caloriesOut': 2363, 'distance': 8.05, 'floors': 10, 'steps': 10000}
2023-04-07
{'activeMinutes': 30, 'caloriesOut': 2363, 'distance': 8.05, 'floors': 

In [115]:
df = pd.DataFrame()

data_fitbit_minutesSedentary = df_fitbit_1(activity[5], base_date, end_date, token)
df['date'] = pd.DataFrame(data_fitbit_minutesSedentary['activities-minutesSedentary'])['dateTime']
df['date'] = pd.to_datetime(df['date'],infer_datetime_format=True)
df['minutesSedentary'] = pd.DataFrame(data_fitbit_minutesSedentary['activities-minutesSedentary'])['value'].astype(int)
data_fitbit_LightlyActive = df_fitbit_1(activity[6], base_date, end_date, token)
df['minutesLightlyActive'] = pd.DataFrame(data_fitbit_LightlyActive['activities-log-minutesLightlyActive'])['value'].astype(float)
data_fitbit_FairlyActive = df_fitbit_1(activity[7], base_date, end_date, token)
df['minutesFairlyActive'] = pd.DataFrame(data_fitbit_FairlyActive['activities-log-minutesFairlyActive'])['value'].astype(int)
data_fitbit_VeryActive = df_fitbit_1(activity[8], base_date, end_date, token)
df['minutesVeryActive'] = pd.DataFrame(data_fitbit_VeryActive['activities-log-minutesVeryActive'])['value'].astype(int)

df

,date,minutesSedentary,minutesLightlyActive,minutesFairlyActive,minutesVeryActive
0,2023-03-29,727,213.0,3,11
1,2023-03-30,776,153.0,6,15
2,2023-03-31,730,152.0,6,26
3,2023-04-01,753,218.0,34,12
4,2023-04-02,745,250.0,26,20
5,2023-04-03,823,74.0,0,0
6,2023-04-04,644,176.0,24,31
7,2023-04-05,553,270.0,9,13
8,2023-04-06,691,204.0,5,18
9,2023-04-07,665,163.0,6,9
